In [1]:
import torch.nn as nn
import torch
from torchinfo import summary
import numpy as np
import os
import coremltools as ct

Torch version 2.2.2 has not been tested with coremltools. You may run into unexpected errors. Torch 2.1.0 is the most recent version that has been tested.


Зададим класс LinearRegressionModel который инициализирует простейшую нейросеть на PyTorch вида линейной модели.

## Класс `LinearRegressionModel`

Этот класс реализует модель линейной регрессии с использованием PyTorch. Он наследует от `nn.Module` и включает базовый линейный слой, который преобразует входные данные из `input_size` в `output_size`.

### Конструктор `__init__`

Инициализирует линейный слой `self.fc`, который принимает размерности входа (`input_size`) и выхода (`output_size`), оба по умолчанию равны 1.

### Метод `forward`

Определяет прямой проход модели, просто возвращая результат применения линейного слоя к входным данным `x`.


In [2]:

class LinearRegressionModel(nn.Module):
    def __init__(self, input_size=1, output_size=1):
        """
        Инициализация простой линейной регрессии.
        """
        super(LinearRegressionModel, self).__init__()
        self.fc = nn.Linear(input_size, output_size)  # Один линейный слой

    def forward(self, x):
        """
        Определение прямого прохода через сеть.
        """
        return self.fc(x)


## Загрузка и подготовка модели к оценке

Загружаем сохраненную модель линейной регрессии из файла `simple_linear_model.pth` с помощью `torch.load`. После загрузки модели вызываем `model.eval()`, чтобы перевести её в режим оценки. Это деактивирует некоторые функции, такие как dropout и пакетная нормализация, обеспечивая консистентное поведение модели при использовании для предсказаний.


In [3]:
model = torch.load("simple_linnear_model.pth")
model.eval()

LinearRegressionModel(
  (fc): Linear(in_features=1, out_features=1, bias=True)
)

## Суммарное описание модели

Используем функцию `summary` для получения детального описания всех слоев модели, включая количество параметров и размеры входных и выходных данных каждого слоя. Это полезно для проверки структуры модели и убеждения в её правильности перед проведением оценки или дополнительным обучением.


In [4]:
summary(model)

Layer (type:depth-idx)                   Param #
LinearRegressionModel                    --
├─Linear: 1-1                            2
Total params: 2
Trainable params: 2
Non-trainable params: 0

## Подготовка входных данных для модели

Создаем тензор `input_data`, содержащий 100 случайно сгенерированных значений, соответствующих нормальному распределению. Каждое значение является одномерным, что соответствует ожидаемому входу модели линейной регрессии. Для проверки содержимого тензора используем `print(torch.tensor(input_data))`, который выводит значения тензора.


In [26]:
input_data = torch.randn(50,1)
input_data = torch.tensor(input_data)
print(input_data)

tensor([[ 1.1774],
        [ 0.3782],
        [-0.0203],
        [-0.6435],
        [-1.1925],
        [-0.0723],
        [ 1.3474],
        [-0.1897],
        [-1.0013],
        [ 0.7301],
        [ 0.2109],
        [-0.9933],
        [ 0.5466],
        [ 0.0633],
        [ 2.1628],
        [-1.8348],
        [-0.9260],
        [ 0.5444],
        [ 0.1601],
        [-0.8776],
        [-1.9140],
        [ 0.7502],
        [ 0.0407],
        [-0.5596],
        [-2.6954],
        [ 0.5877],
        [ 0.2613],
        [ 0.4208],
        [-1.9816],
        [ 0.5083],
        [-0.4530],
        [ 0.3650],
        [ 0.3014],
        [ 0.1514],
        [-0.7997],
        [-1.3751],
        [-0.7417],
        [ 1.4595],
        [-0.3557],
        [-0.7295],
        [-1.0353],
        [ 0.1886],
        [-0.1032],
        [-0.8746],
        [ 0.1787],
        [ 0.2055],
        [-0.4063],
        [ 0.9656],
        [ 0.7149],
        [-1.7231]])


/var/folders/2m/1nfzrjzx5rjdbx6qbmzt0bjw0000gp/T/ipykernel_71482/298862586.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_data = torch.tensor(input_data)


## Подготовка и использование входных данных для предсказания

### Подготовка входных данных

В этом разделе мы создаем тензор `input_data` с заданными значениями. Эти данные представляют собой вектор признаков, которые будут использоваться для получения предсказаний от модели линейной регрессии. Данные включают в себя различные числовые значения, распределенные в колонке.

### Выполнение предсказаний

Следующий шаг — использование модели для получения предсказаний на основе входных данных. Мы используем контекст `torch.no_grad()`, чтобы отключить расчет градиентов, что является стандартной практикой при делании предсказаний, поскольку это снижает потребление памяти и ускоряет процесс. Затем мы применяем модель к `input_data` и сохраняем результат в переменной `predictions`.



In [27]:
with torch.no_grad():
    predictions = model(input_data)


## Сохранение предсказаний в файл

### Задание пути к файлу

Переменная `file_path` задает имя файла, в который будут сохранены предсказания модели. Имя файла `LIN_значение.txt` указывает на то, что файл содержит значения линейной регрессии.

### Сохранение данных

Используя функцию `np.savetxt` из библиотеки NumPy, сохраняем предсказания модели в текстовый файл. Метод `flatten` применен к массиву `predictions` для преобразования его в одномерный массив, что упрощает запись данных. Аргумент `delimiter=', '` определяет, что значения в файле будут разделены запятой и пробелом.

Этот подход обеспечивает удобное хранение и возможность последующего анализа результатов модели.


In [7]:
current_directory = os.getcwd()
parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))

file_path = f'{parent_directory}/Files/LIN_значение.txt'

np.savetxt(file_path, predictions.flatten(), delimiter=', ')


## Сохранение модели в формате TorchScript

### Подготовка примера входных данных

Первым шагом является подготовка `input_trace_example`, примера входных данных, который необходим для трассировки модели. В данном случае создается тензор случайных данных размером `(100, 1)`, что соответствует ожидаемым размерам входа модели.

### Трассировка модели

Функция `torch.jit.trace` используется для создания трассированной версии модели. Этот процесс включает выполнение модели с примером входных данных, что позволяет TorchScript записать операции, выполняемые моделью. Результатом является оптимизированная версия модели, которая может быть эффективно выполнена на различных платформах.

### Сохранение трассированной модели

После трассировки модель сохраняется в файл `LIN_model_traced.pt` с использованием `torch.jit.save`. Этот формат файлов TorchScript удобен для последующего развертывания модели в различных средах, включая серверные и мобильные платформы.

Этот подход позволяет улучшить переносимость и производительность модели, обеспечивая возможность её использования в разнообразных приложениях.


In [36]:
input_trase_example = (torch.randn(50,1))
traced_model = torch.jit.trace(model, input_trase_example)
torch.jit.save(traced_model, f'{parent_directory}/Files/LIN_model_traced.pt')

In [37]:
LIN_loaded_model = torch.jit.load(f'{parent_directory}/Files/LIN_model_traced.pt')
LIN_loaded_model.eval()

RecursiveScriptModule(
  original_name=LinearRegressionModel
  (fc): RecursiveScriptModule(original_name=Linear)
)

In [38]:
summary(LIN_loaded_model)

Layer (type:depth-idx)                   Param #
LinearRegressionModel                    --
├─Linear: 1-1                            2
Total params: 2
Trainable params: 2
Non-trainable params: 0

In [43]:
LIN_example_input = torch.randn(50, 1)

In [44]:
mlmodel = ct.convert(
    LIN_loaded_model,
    inputs=[ct.TensorType(shape=LIN_example_input.shape)],
    convert_to='neuralnetwork'
)

'''Можно запустить модель для проверки перед сохранением'''


# Сохраняем модель в файл
mlmodel.save(f"{parent_directory}/Files/LIN_CoreML.mlmodel")

Support for converting Torch Script Models is experimental. If possible you should use a traced model for conversion.
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 3/3 [00:00<00:00, 13400.33 ops/s]


In [45]:
LIN_coreML = ct.models.MLModel(f'{parent_directory}/Files/LIN_CoreML.mlmodel')

In [46]:
spec = LIN_coreML.get_spec()
print(spec.description.input)
print(spec.description.output)

[name: "x_1"
type {
  multiArrayType {
    shape: 50
    shape: 1
    dataType: FLOAT32
  }
}
]
[name: "linear_0"
type {
  multiArrayType {
    dataType: FLOAT32
  }
}
]


In [47]:
input_d = {'x_1':input_data}
LIN_predict = LIN_coreML.predict(input_d)

In [51]:
a = LIN_predict['linear_0'].flatten()

print(f"Предсказание PyTorch - {predictions}")
print(f"\nПредсказание CoreML - {a}")

AttributeError: 'Tensor' object has no attribute 'tensor'